In [1]:
import os
import cv2
from ultralytics import YOLO

In [2]:
# Percorsi ai modelli YOLO
MODEL_PATH_ARMI = "/home/poli/Tesi_Progetto/pipeline/models/best_weapons.pt"
MODEL_PATH_SIMBOLI = "/home/poli/Tesi_Progetto/pipeline/models/best_flags.pt"
MODEL_PATH_PERSONE = "/home/poli/Tesi_Progetto/pipeline/models/best_people.pt"

# Directory immagini di test
IMAGES_DIR = "images/"
OUTPUT_DIR = "output/"

# Crea la directory di output
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:


# Caricamento dei modelli
model_armi = YOLO(MODEL_PATH_ARMI)
model_simboli = YOLO(MODEL_PATH_SIMBOLI)
model_persone = YOLO(MODEL_PATH_PERSONE)

print("Modelli caricati con successo!")


In [4]:
def process_image(image_path):
    """
    Processa un'immagine attraverso i modelli YOLO e restituisce i rilevamenti (simboli, persone, armi).
    """
    # Carica l'immagine
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Immagine non trovata: {image_path}")

    # Esegui i modelli
    results_armi = model_armi.predict(image)
    results_simboli = model_simboli.predict(image)
    results_persone = model_persone.predict(image)

    # Combina i risultati
    detections = extract_detections(results_armi, model_armi) + \
                 extract_detections(results_simboli, model_simboli) + \
                 extract_detections(results_persone, model_persone)

    return detections






In [5]:
def extract_detections(results, model):
    """
    Estrae sia i bounding box che le segmentazioni dai risultati di un modello YOLO.
    """
    detections = []

    # Bounding box
    if results[0].boxes:
        for box in results[0].boxes:
            detections.append({
                "type": "box",
                "label": model.names[int(box.cls[0])],
                "confidence": float(box.conf[0]),
                "coordinates": box.xyxy[0].tolist()  # Bounding box coordinates
            })

    return detections


In [6]:
def filter_detections(detections, class_thresholds, global_threshold=0.5):
    """
    Filtra i rilevamenti in base alle soglie di confidenza.
    :param detections: Lista di rilevamenti.
    :param class_thresholds: Dizionario delle soglie specifiche per classe.
    :param global_threshold: Soglia globale, usata in assenza di valori per classe.
    :return: Lista di rilevamenti filtrati.
    """
    filtered = []
    for det in detections:
        label = det["label"]
        confidence = det["confidence"]

        # Usa la soglia specifica per la classe, altrimenti quella globale
        threshold = class_thresholds.get(label, global_threshold)
        if confidence >= threshold:
            filtered.append(det)
    return filtered


In [7]:
class_thresholds = {
    "Terrorist_symbol": 0.25,
    "NoTerrorist_symbol": 0.25,
    "Terrorist": 0.4,
    "NoTerrorist": 0.4,
    "gun": 0.45,
    "grenade": 0.45,
    "bazooka": 0.4,
    "knife": 0.4,
    "pistol": 0.4,
}



In [8]:
def classify_context_with_weights(detections):
    """
    Classifica il contesto basandosi su simboli, persone e armi, con pesatura delle armi applicata in ogni contesto.
    """
    # Inizializza i conteggi
    count_terrorist_symbol = 0
    count_no_terrorist_symbol = 0
    count_terrorist_person = 0
    count_no_terrorist_person = 0
    weapon_scores = {"terrorist": 0, "no_terrorist": 0}

    # Processa i rilevamenti
    for det in detections:
        label = det["label"]
        confidence = det["confidence"]

        # Simboli
        if label == "Terrorist_symbol":
            count_terrorist_symbol += 1
        elif label == "NoTerrorist_symbol":
            count_no_terrorist_symbol += 1

        # Persone
        elif label in "Terrorist":
            count_terrorist_person += 1
        elif label == "NoTerrorist":
            count_no_terrorist_person += 1

        # Armi con pesi
        elif label in ["Gun", "Grenade", "Bazooka", "Knife", "Pistol"]:
            if label in ["Pistol", "Grenade","Knife"]:
                weapon_scores["no_terrorist"] += confidence * 1.5  # Peso maggiore per no terrorist
            elif label in ["Bazooka", "Gun"]:
                weapon_scores["terrorist"] += confidence * 1.5  # Peso maggiore per terrorist

    # Debug: stampa conteggi
    print(f"Simboli Terroristici: {count_terrorist_symbol}, Simboli No Terroristici: {count_no_terrorist_symbol}")
    print(f"Weapon Scores: {weapon_scores}")

    # Logica di classificazione
    if count_terrorist_symbol > 0 and count_no_terrorist_symbol == 0:
        # Solo simboli terroristici
        return "Terroristico"
    elif count_no_terrorist_symbol > 0 and count_terrorist_symbol == 0:
        # Solo simboli non terroristici
        return "No Terroristico"
    elif count_terrorist_symbol > 0 or count_no_terrorist_symbol > 0:
        # Simboli e armi rilevati
        if weapon_scores["terrorist"] > weapon_scores["no_terrorist"]:
            return "Terroristico"
        elif weapon_scores["no_terrorist"] > weapon_scores["terrorist"]:
            return "No Terroristico"
        else:
            return "Ambiguo"
    elif count_terrorist_symbol > 0 and count_terrorist_person > 0:
        # Simboli terroristici e persone terroristiche
        return "Terroristico"
    elif count_terrorist_symbol == 0 and count_no_terrorist_symbol == 0 and count_terrorist_person > 0:
        if weapon_scores["terrorist"] > weapon_scores["no_terrorist"]:
            return "Terroristico"
        return "No Terroristico"
    elif count_terrorist_symbol == 0 and count_no_terrorist_symbol == 0 and count_no_terrorist_person > 0:
        if weapon_scores["no_terrorist"] > weapon_scores["terrorist"]:
            return "No Terroristico"
        return "Ambiguo"
    elif count_terrorist_symbol == 0 and count_no_terrorist_symbol == 0 and weapon_scores["terrorist"] > weapon_scores["no_terrorist"]:
        return "Terroristico"
    elif count_terrorist_symbol == 0 and count_no_terrorist_symbol == 0 and weapon_scores["no_terrorist"] > weapon_scores["terrorist"]:
        return "No Terroristico"
    else:
        return "Ambiguo"


In [ ]:
for filename in os.listdir(IMAGES_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(IMAGES_DIR, filename)
        print(f"Processando {filename}...")

        # Processa l'immagine
        detections = process_image(image_path)

        # Filtra i rilevamenti
        filtered_detections = filter_detections(detections, class_thresholds)

        # Classifica il contesto
        context = classify_context_with_weights(filtered_detections)

        print(f"Immagine: {filename}, Contesto rilevato: {context}")



In [ ]:
import pandas as pd

results = []
for filename in os.listdir(IMAGES_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(IMAGES_DIR, filename)

        detections = process_image(image_path)
        filtered_detections = filter_detections(detections, class_thresholds)
        context = classify_context_with_weights(filtered_detections)

        results.append({"filename": filename, "context": context})

# Salva in un CSV
df_results = pd.DataFrame(results)
df_results.to_csv(os.path.join(OUTPUT_DIR, "classification_results.csv"), index=False)
print("Risultati salvati in classification_results.csv")



In [11]:
import cv2

def process_and_visualize_image(image_path, detections, output_dir):
    """
    Processa un'immagine e disegna i bounding box sui rilevamenti.
    :param image_path: Percorso dell'immagine originale.
    :param detections: Lista di rilevamenti.
    :param output_dir: Directory di output per salvare l'immagine annotata.
    """
    # Carica l'immagine originale
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Immagine non trovata: {image_path}")

    # Colori per le categorie
    colors = {
        "Gun": (0, 255, 0),  # Verde
        "Grenade": (255, 0, 0),  # Blu
        "Bazooka": (0, 0, 255),  # Rosso
        "Knife": (255, 255, 0),  # Giallo
        "Pistol": (0, 255, 255),  # Ciano
        "Terrorist_symbol": (128, 0, 128),  # Viola
        "NoTerrorist_symbol": (0, 128, 128),  # Turchese
        "Terrorist": (128, 128, 0),  # Marrone
        "NoTerrorist": (128, 128, 128),  # Grigio
    }

    # Disegna i bounding box
    for det in detections:
        label = det["label"]
        confidence = det["confidence"]
        coordinates = det.get("coordinates", None)  # Coordinate del bounding box

        if coordinates:
            x1, y1, x2, y2 = map(int, coordinates)
            color = colors.get(label, (255, 255, 255))  # Bianco per label sconosciuti
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            text = f"{label} ({confidence:.2f})"
            cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Salva l'immagine annotata
    output_path = f"{output_dir}/{os.path.basename(image_path)}"
    cv2.imwrite(output_path, image)
    print(f"Immagine annotata salvata in: {output_path}")


In [ ]:
# Directory di output per immagini annotate
ANNOTATED_DIR = "annotated_images/"
os.makedirs(ANNOTATED_DIR, exist_ok=True)

for filename in os.listdir(IMAGES_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(IMAGES_DIR, filename)
        print(f"Processando {filename}...")

        # Processa l'immagine e ottieni i rilevamenti
        detections = process_image(image_path)

        # Disegna i bounding box
        process_and_visualize_image(image_path, detections, ANNOTATED_DIR)
